In [54]:
import tushare as ts
import pandas as pd
import matplotlib.pyplot as plt
pro = ts.pro_api('c1dfcd419324efa15b3b2269de1180befae1562f928410e3900cfe04')
from matplotlib.font_manager import FontProperties
font = FontProperties(fname="SimHei.ttf", size=14)

1、获取一日的不同类型交易者的主动买入占比

In [55]:
def get_daily_buy_per(trade_date):
    """
    获取一日的不同类型交易者的主动买入占比
    :param trade_date: 交易日期
    :return: 三种交易者的主动买入占比额
    """
    # 获取单日全部股票数据（单位：万元）
    data = pro.moneyflow(trade_date=trade_date)
    data.set_index('ts_code', drop=True, inplace=True)
    data['大单买入'] = data['buy_elg_amount']
    data['中单买入'] = data['buy_lg_amount']
    data['小单买入'] = data['buy_sm_amount'] + data['buy_md_amount']
    data = data.loc[:, ['大单买入', '中单买入', '小单买入']]
    sm_net_buy = data.sum()['小单买入'] / data.sum().sum()
    md_net_buy = data.sum()['中单买入'] / data.sum().sum()
    bg_net_buy = data.sum()['大单买入'] / data.sum().sum()
    buy_sm_amount = data.sum()['小单买入']
    buy_lg_amount = data.sum()['中单买入']
    buy_elg_amount = data.sum()['大单买入']

    return sm_net_buy, md_net_buy, bg_net_buy, buy_sm_amount, buy_lg_amount, buy_elg_amount

2、获取开盘的日期

In [56]:
date_list_df = pro.trade_cal(exchange='SSE', start_date='20180101', end_date='20230401')
date_list_df = date_list_df[date_list_df['is_open'] == 1]
date_list = list(date_list_df['cal_date'])

3、循环日期，获取每日占比

In [57]:
df = pd.DataFrame(columns=['小单主动买入占比', '中单主动买入占比', '大单主动买入占比', '小单买入额(万元)', '中单买入额(万元)','大单买入额(万元)'])

for date in date_list:
    sm_net_buy, md_net_buy, bg_net_buy , buy_sm_amount, buy_lg_amount, buy_elg_amount= get_daily_buy_per(date)
    df.loc[date]=[sm_net_buy, md_net_buy, bg_net_buy, buy_sm_amount, buy_lg_amount, buy_elg_amount]

df

In [58]:
df = df.reset_index(drop=False)
df = df.rename(columns={'index':'交易日期'})
df = df.sort_values(by='交易日期', ascending=True)
df['交易日期'] = pd.to_datetime(df['交易日期'])
df = df.set_index('交易日期')
df.to_csv('主动买入成交占比.csv', encoding='utf-8', index=True)

4、可视化

In [61]:
plt.figure(figsize=(20, 4))
plt.plot(df.iloc[:, :3])
plt.show()

4、获取上证指数数据

In [60]:
index_df = pro.index_daily(ts_code='000001.SH', start_date='20180101', end_date='20230401')
index_df.to_csv('SSE.csv', index=False)